# 모델기반 추천 시스템의 개요

## 모델기반 협업 필터링

- 머신러닝을 가장 잘 활용한 추천알고리즘의 일종
- 주어진 데이터를 활용하여 모델이 패턴을 학습
- 데이터 크기 또는 특징을 동적으로 활용 가능
- 데이터의 잠재적 특성을 파악하는 모델(Latent Factor Model)

## 모델기반 협업 필터링의 장점

- 추천모델의 크기를 줄일 수 있음(데이터를 Matrix로 만들어서 가능)
- 데이터 전처리와 학습과정으로 준비된 모델로 예측 가능함
- 추천모델의 과적합 방지가 가능함

## 모델기반 협업 필터링 종류

- Association Rule Mining
- Matrix Factorization(SVD, ALS)
- Probabilistic Models
- SVM, Regression methods(Logistic Regression), Deep Learning

# 특이값 분해 추천 시스템(SVD) 정리

## SVD(Singular Vector Decomposition)

- 일반적으로 잘 알려진 행렬 분할 방법으로, 강력한 성능을 자랑하여 아직까지도 많은 곳에서 사용하고 있는 모델이다.
- 본래 고유값 분해(Eigen Value Decomposition)를 이용한 행렬분해의 경우, 정방행렬(n x n)에 이어야 하지만, SVD를 이용한다면 m x n 행렬로도 대각행렬을 통한 특이값 분해가 가능하기 때문에 유저, 아이템의 행과 열의 개수가 다른 추천모델에도 적합하다.
- 고유값 분해
    - n x n 의 정방행렬에 대해 선형변환 후의 벡터가 얼마나 커지거나 작아졌는지를 파악하기 위한 방법이자, 3개의 행렬로 분할하는 방법이다.
- 특이값 분해
    - SVD는 특이값 분해로, m x n으로 이뤄진 A행렬에 대한 행렬분해를 하는 것을 의미한다.
기하학에서 선형변환 시 여전히 직교하지만 얼마만큼의 크기 변화가 있는지를 파악하는 것을 의미한다.

## Numpy로 SVD 구현

In [1]:
import numpy as np
from numpy.linalg import svd

In [2]:
# 4 x 4 행렬 생성
np.random.seed(42)
a = np.random.randn(4, 4)  # 평균 0, 표준편차 1의 가우시간 난수 생성
print(np.round(a, 3))

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [-0.469  0.543 -0.463 -0.466]
 [ 0.242 -1.913 -1.725 -0.562]]


In [3]:
# 이렇게 임의의 4 x 4 매트릭스를 생성한 뒤,
# Numpy SVD를 통해3개의 행렬로 분할해본다.
U, Sigma, Vt = svd(a)
print('U:', np.round(U, 3))
print('Sigma:',
      np.round(Sigma, 3))  # 시그마의 경우, 대각행렬의 요소값인 Singular Value를 내림차순으로 가져온다.
print('Vt:', np.round(Vt, 3))

U: [[-0.373 -0.598  0.642 -0.302]
 [-0.488 -0.35  -0.745 -0.289]
 [ 0.113  0.444  0.062 -0.887]
 [ 0.781 -0.568 -0.168 -0.197]]
Sigma: [3.08  1.926 0.92  0.342]
Vt: [[ 0.021 -0.412 -0.783 -0.466]
 [-0.291  0.775 -0.086 -0.554]
 [ 0.461  0.479 -0.544  0.512]
 [ 0.838  0.017  0.289 -0.462]]


In [4]:
# 시그마의 대각행렬을 원상복구 해주는 방법: np.diag
Sigma_matrix = np.diag(Sigma)
print('Sigma :', np.round(Sigma_matrix, 3))

Sigma : [[3.08  0.    0.    0.   ]
 [0.    1.926 0.    0.   ]
 [0.    0.    0.92  0.   ]
 [0.    0.    0.    0.342]]


In [5]:
a_ = np.dot(np.dot(U, Sigma_matrix), Vt)
print(np.round(a, 3))  # 맨 처음의 매트릭스 a와 같음을 확인할 수 있다.

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [-0.469  0.543 -0.463 -0.466]
 [ 0.242 -1.913 -1.725 -0.562]]


## sklearn을 이용해 영화 추천 구현

In [42]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split

In [34]:
# 패키지 내에서 유용하게 사용할 수 있는 무비렌즈 데이터 불러오기
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [43]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [35]:
model = SVD()

In [36]:
cross_validate(model, data, measures=['rmse', 'mse'], cv=5, verbose=True)

Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9347  0.9419  0.9382  0.9318  0.9350  0.9363  0.0034  
MSE (testset)     0.8737  0.8871  0.8802  0.8682  0.8742  0.8767  0.0064  
Fit time          2.65    2.66    2.62    2.64    2.65    2.64    0.01    
Test time         0.08    0.07    0.07    0.15    0.07    0.09    0.03    


{'test_rmse': array([0.93473125, 0.94185324, 0.93816438, 0.93178786, 0.93500753]),
 'test_mse': array([0.8737225 , 0.88708752, 0.88015241, 0.86822861, 0.87423909]),
 'fit_time': (2.6533565521240234,
  2.6630873680114746,
  2.620145082473755,
  2.639962673187256,
  2.6477153301239014),
 'test_time': (0.08394169807434082,
  0.06861495971679688,
  0.06999588012695312,
  0.14923334121704102,
  0.06807947158813477)}

In [44]:
model.fit(trainset)

In [46]:
# uid : 유저 아이디
# iid : 아이템 아이디(영화 아이디)
# r_ui : 실제 평점
# est : 예측 평점
predictions = model.test(testset)
print('prediction type : ', type(predictions), 'size : ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'> size :  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.599439175192738, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.4401446990224875, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.880640926021253, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6788704821857974, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.6020911932053616, details={'was_impossible': False})]

In [47]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.599439175192738),
 ('882', '291', 3.4401446990224875),
 ('535', '507', 3.880640926021253)]

In [49]:
uid = str(196)
iid = str(302)
pred = model.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.16   {'was_impossible': False}


# 추천 시스템 사례

## 카카오

- https://tech.kakao.com/2021/03/11/kakao-ai

## 네이버

- https://m.blog.naver.com/naver_diary/220936643956

## 넷플릭스

- https://brunch.co.kr/@cysstory/159

## 멜론

- https://brunch.co.kr/@kakao-it/342